In [13]:
import numpy 
import torch
import json
import pandas
import re

from tmdbv3api import TMDb
from tmdbv3api import Movie

tmdb = TMDb()
tmdb.api_key = 'ecbe5b50079424c4372da15d2999da46'
tmdb.language = 'en'
tmdb.debug = True

movie = Movie()

In [14]:
"""his script returns four list:
1. all_movie_db: ID to descripiton (plot) using all movies in the IMDb
2. metioned_movie_db: ID to descripiton using the movies mentioned in redial (include both train and test)
3. all_data_train: train data
4. all_data_test: test data

5. not_exist_metioned_movie_db
"""

'his script returns four list:\n1. all_movie_db: ID to descripiton (plot) using all movies in the IMDb\n2. metioned_movie_db: ID to descripiton using the movies mentioned in redial (include both train and test)\n3. all_data_train: train data\n4. all_data_test: test data\n\n5. not_exist_metioned_movie_db\n'

## movie database

In [15]:
# IMDb dataset

df = pandas.read_csv("IMDb.csv")

# our ID to description
all_movie_db = {} # ID to descripiton
imdb_name_with_year_to_id = {}
imdb_name_without_year_to_id = {}

# unique imdb id set
imdb_id_set = {}

# unique duplicate name set
duplicate_name_set = {}

key = 0
for i in range(len(df)):
    mid = df['imdb_title_id'][i]
    if mid not in imdb_id_set.keys():
        imdb_id_set[mid]=1
        name = df['original_title'][i].lower()
        
        top3_actors = "nan" if str(df['actors'][i]) == "nan" else ','.join(df['actors'][i].split(',')[:3])
        director  = df['director'][i]
        genres = df['genre'][i]
        
        year = df['year'][i]
        full_name = name + ' (' + str(year) + ')'
        desc = df['description'][i]
        
        concated_meta = str(name) + ' [SEP] ' + str(top3_actors) + ' [SEP] ' + \
            str(director) + ' [SEP] ' + str(genres) + ' [SEP] ' + str(desc)
    
        if name in imdb_name_without_year_to_id.keys():
            duplicate_name_set[name] = 1
        
        # for movie match in redial dataset
        all_movie_db[key] = concated_meta
        imdb_name_with_year_to_id[full_name] = key
        imdb_name_without_year_to_id[name] = key
        key +=1 

# Movies_metadata dataset
new_count = 0
df = pandas.read_csv("movies_metadata.csv")
for i in range(len(df)):
    mid = df['imdb_id'][i]
    if mid not in imdb_id_set.keys():
        new_count += 1
        imdb_id_set[mid]=1
        name = df['original_title'][i].lower()
        
        genres = json.loads(df["genres"][i].replace('\'', '\"'))
        genres = ', '.join([ g['name'] for g in genres])
        
        year = df['release_date'][i]
        year = re.findall(r"\d\d\d\d", str(year))
        if len(year) > 0:
            year = year[0]
            full_name = name + ' (' + str(year) + ')'
        else:
            full_name = name
        desc = df['overview'][i]
        
        concated_meta = str(name) + ' [SEP] [SEP] [SEP] ' + str(genres) + ' [SEP] ' + str(desc)
        
        if name in imdb_name_without_year_to_id.keys():
            duplicate_name_set[name] = 1
        
        # for movie match in redial dataset
        all_movie_db[key] = concated_meta
        imdb_name_with_year_to_id[full_name] = key
        imdb_name_without_year_to_id[name] = key
        key +=1 

# Inspired dataset
new_count2 = 0       
df = pandas.read_csv("movie_database.tsv", sep = '\t')
for i in range(len(df)):
    mid = df['imdb_id'][i]
    if mid not in imdb_id_set.keys():
        new_count2+=1
        imdb_id_set[mid]=1
        name = df['title'][i].lower()
        year = df['year'][i]
        desc = df['short_plot'][i]
        full_name = name + ' (' + str(year) + ')'
        if name in imdb_name_without_year_to_id.keys():
            duplicate_name_set[name] = 1
            
        concated_meta = str(name) + " [SEP] " + \
              str(df['actors'][i]) + " [SEP] " + \
              str(df['director'][i]) + " [SEP] " + \
              str(df['genre'][i]) + " [SEP] " + \
              str(desc)

        # for movie match in redial dataset
        all_movie_db[key] = concated_meta
        imdb_name_with_year_to_id[full_name] = key
        imdb_name_without_year_to_id[name] = key
        key +=1
        
# sparql responses
sparql_dic = torch.load("/local-scratch1/data/by2299/sparql_processed_dic")

for k, v in sparql_dic.items():
    all_movie_db[key] = v
    if '(' in k:
        imdb_name_with_year_to_id[k.lower()] = key
        imdb_name_without_year_to_id[k.split('(')[0].strip().lower()] = key
    else:
        imdb_name_without_year_to_id[k.lower()] = key
    key += 1

In [16]:
print(len(imdb_name_with_year_to_id.keys()))
print(len(imdb_name_without_year_to_id.keys()))
print(len(duplicate_name_set.keys()))
print(new_count, new_count2)

97694
91503
4600
11345 672


In [17]:
test_data = []
for line in open("./website-data/redial_dataset/test_data.jsonl", "r"):
    test_data.append(json.loads(line))
print("Loaded {} test conversations".format(len(test_data)))

Loaded 1342 test conversations


In [18]:
test_data[0]

{'movieMentions': {'111776': 'Super Troopers (2001)',
  '91481': 'Beverly Hills Cop (1984)',
  '151656': 'Police Academy  (1984)',
  '134643': 'American Pie  (1999)',
  '192131': 'American Pie ',
  '124771': '48 Hrs. (1982)',
  '94688': 'Police Academy 2: Their First Assignment (1985)',
  '101794': 'Lethal Weapon (1987)'},
 'respondentQuestions': {'111776': {'suggested': 0, 'seen': 1, 'liked': 1},
  '91481': {'suggested': 1, 'seen': 2, 'liked': 2},
  '151656': {'suggested': 1, 'seen': 0, 'liked': 1},
  '134643': {'suggested': 0, 'seen': 1, 'liked': 1},
  '192131': {'suggested': 0, 'seen': 1, 'liked': 1},
  '124771': {'suggested': 1, 'seen': 2, 'liked': 2},
  '94688': {'suggested': 1, 'seen': 0, 'liked': 1},
  '101794': {'suggested': 1, 'seen': 0, 'liked': 2}},
 'messages': [{'timeOffset': 0,
   'text': 'Hi I am looking for a movie like @111776',
   'senderWorkerId': 956,
   'messageId': 204171},
  {'timeOffset': 48,
   'text': 'You should watch @151656',
   'senderWorkerId': 957,
   'm

In [19]:
def refine_movie_name(old_name):
    """
    regularize move name to match the database. default is name + space + (year), if without year, return name only
    return (new name, flag)
    flag 0 means without year
    flag 1 means with year
    """
    year = re.findall(r"\(\d\d\d\d\)", old_name)
    if len(year) > 0:
        year = year[0]
        name = old_name.split(year)[0]
        movie_name = name.strip() + ' ' + year
        return (movie_name, 1)
    else:
        movie_name = old_name.strip()
        return (movie_name, 0)

In [20]:
# def movie_search(mv_name_with_year_to_id, mv_name_without_year_to_id, movie_name):
#     movie_name, year_flag = refine_movie_name(movie_name)
    
#     if year_flag == 1 and movie_name.lower() in mv_name_with_year_to_id.keys():
#         our_id = mv_name_with_year_to_id[movie_name.lower()]

#     elif year_flag == 0 and movie_name.lower() in imdb_name_without_year_to_id.keys():  
#         our_id = imdb_name_without_year_to_id[movie_name.lower()]
#         if movie_name.lower() in duplicate_name_set.keys():
#             mentioned_is_in_duplicate += 1
            
#     elif year_flag == 1 and movie_name.split(' (')[0].lower() in mv_name_without_year_to_id.keys():
#         our_id = mv_name_without_year_to_id[movie_name.split(' (')[0].lower()]
#         if movie_name.split(' (')[0].lower() in duplicate_name_set.keys():
#             mentioned_is_in_duplicate += 1
#     else:
#         our_id = None
#         not_exist_count += 1
#         not_exist_metioned_movie_db[movie_name] = 1

In [25]:
metioned_movie_db = {} # ID to descripiton
not_exist_metioned_movie_db = {}
all_mentioned_movie = {} # unique movie mentioned
mentioned_is_in_duplicate = 0

mention_count = 0 # can contain duplicate
not_exist_count = 0 # movie not in the database

test_data = []
for line in open("./website-data/redial_dataset/test_data.jsonl", "r"):
    test_data.append(json.loads(line))
print("Loaded {} test conversations".format(len(test_data)))

train_data = []
for line in open("./website-data/redial_dataset/train_data.jsonl", "r"):
    train_data.append(json.loads(line))
print("Loaded {} train conversations".format(len(train_data)))

all_data = train_data + test_data

for i in range(len(all_data)):
    temp = all_data[i]
    mentioned_movies = temp['movieMentions']
    if len(mentioned_movies) == 0:
        continue
    for key in list(mentioned_movies.keys()):
        movie_name = mentioned_movies[key]
        if movie_name is None:
            continue
        movie_name, year_flag = refine_movie_name(movie_name)
        all_mentioned_movie[movie_name] = 1
        mention_count += 1
        
        if year_flag == 1 and movie_name.lower() in imdb_name_with_year_to_id.keys():
            our_id = imdb_name_with_year_to_id[movie_name.lower()]
            metioned_movie_db[our_id] = all_movie_db[our_id]
            
        elif year_flag == 0 and movie_name.lower() in imdb_name_without_year_to_id.keys():  
            our_id = imdb_name_without_year_to_id[movie_name.lower()]
            metioned_movie_db[our_id] = all_movie_db[our_id]
            if movie_name.lower() in duplicate_name_set.keys():
                mentioned_is_in_duplicate += 1
                
        elif year_flag == 1 and movie_name.split(' (')[0].lower() in imdb_name_without_year_to_id.keys():
            our_id = imdb_name_without_year_to_id[movie_name.split(' (')[0].lower()]
            metioned_movie_db[our_id] = all_movie_db[our_id]
            if movie_name.split(' (')[0].lower() in duplicate_name_set.keys():
                mentioned_is_in_duplicate += 1
        else:
            not_exist_count += 1
            not_exist_metioned_movie_db[movie_name] = 1

Loaded 1342 test conversations
Loaded 10006 train conversations


In [26]:
print(len(all_mentioned_movie.keys()), len(metioned_movie_db.keys()), len(not_exist_metioned_movie_db.keys()))
print(mention_count, not_exist_count, mentioned_is_in_duplicate)

6629 6222 180
60070 1171 1443


In [23]:
def meta_with_id(m_id, movie_name_no_year):
    movie_deets = movie.details(m_id)
    
    #actors
    actors = ''
    for i, cast in enumerate(movie_deets['casts']['cast']):
        if i <= 1:
            actors += cast['name'] + ', '
        else:
            actors += cast['name']
            break
    
    # genre
    genres = ''
    for i, genre in enumerate(movie_deets["genres"]):
        genres += genre['name'] + ', '
        
    # overview
    overview = movie_deets['overview']
    
    return movie_name_no_year + ' [SEP] ' + actors + ' [SEP] [SEP] ' + genres + ' [SEP] ' + overview

In [24]:
for key in not_exist_metioned_movie_db.keys():
    our_id = len(all_movie_db)
    movie_name, year_flag = refine_movie_name(key)
    movie_name_no_year = None
    if year_flag == 1:
        movie_name_no_year = movie_name.split(' (')[0].lower()
    else:
        movie_name_no_year = movie_name
    if movie_name_no_year == "": continue
    results = movie.search(movie_name_no_year)
    if len(results) == 1:
        all_movie_db[our_id] = meta_with_id(results[0]['id'], movie_name_no_year) #results[0]["overview"]
        imdb_name_with_year_to_id[movie_name.lower()] = our_id
        imdb_name_without_year_to_id[movie_name_no_year.lower()] = our_id
        continue
    valid = False
    for result in results:
        o_title = result['original_title']
        title = result['title']
        # year = result['release_date'].split('-')[0]
        if o_title.lower() == movie_name_no_year.lower() or title.lower() == movie_name_no_year.lower():
            valid = True
            
            m_id = result['id']
            
            all_movie_db[our_id] = meta_with_id(m_id, movie_name_no_year)
            imdb_name_with_year_to_id[movie_name.lower()] = our_id
            imdb_name_without_year_to_id[movie_name_no_year.lower()] = our_id
            break
            

In [17]:
# import csv

# with open('unfound_movies.csv', mode='w') as unfound_file:
#     employee_writer = csv.writer(unfound_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

#     for k, v in not_exist_metioned_movie_db.items():
#         employee_writer.writerow([str(k)])


In [27]:
multi_rec_count = 0 # more than one rec in on sentence
not_label_count = 0 # movie without {suggested} labels
suggest_not_exist_count = 0 # movie not in the database
suggested_is_in_duplicate = 0
not_exist_suggested_movie_db = {}

rec_count = 0

## test data

In [28]:
test_data = []
for line in open("./website-data/redial_dataset/test_data.jsonl", "r"):
    test_data.append(json.loads(line))
print("Loaded {} test conversations".format(len(test_data)))

all_data_test = [] #result

for i in range(len(test_data)):
    temp = test_data[i]
    mentioned_movies = temp['movieMentions']
    status_movies = temp['respondentQuestions']
    messages = temp['messages']
    
    # A is recommender B is customer
    AB_dict = {}
    AB_dict[temp['respondentWorkerId']] = 'A'
    AB_dict[temp['initiatorWorkerId']] = 'B'

    # dialog temp
    this_dialog_tmp = []
    for j in range(len(messages)):
        message = messages[j]
        userID = message['senderWorkerId']
        text = message['text']
        this_dialog_tmp.append([userID, text])

    # merge consecutive sentences from the same user into one sentence
    this_dialog = []
    j = 0
    while j < len(this_dialog_tmp):
        userID = this_dialog_tmp[j][0]
        text = this_dialog_tmp[j][1]
        k = 1
        while j + k < len(this_dialog_tmp) and userID == this_dialog_tmp[j+k][0]:
            text = text + ' ' + this_dialog_tmp[j+k][1]
            k += 1
        j = j + k   
        this_dialog.append(AB_dict[userID] + ": " + text) 
        
    # extract @movie ID, check which is recommendaiton
    this_dialog_post = []

    for item in this_dialog:
        this_setence_movie_id = re.findall(r"@\d+", item)
        recommended_list = []
        for m in this_setence_movie_id:
            m_id = m.split('@')[1]
            
            if len(status_movies) > 0 and m_id in status_movies.keys():# and status_movies[m_id]['suggested']==1:
                recommended_list.append(m_id)
            
            # replace id with name in the dialog
            movie_name = mentioned_movies[m_id]
            movie_name, year_flag = refine_movie_name(movie_name)
#             item = item.replace(str(m), movie_name)
            item = item.replace(str(m), "[MOVIE_ID]") # use placeholder
        our_ids = []
        if len(recommended_list) > 0:
            for rec_id in range(len(recommended_list)):
                movie_name = mentioned_movies[recommended_list[rec_id]]
                movie_name, year_flag = refine_movie_name(movie_name)
                if year_flag == 1 and movie_name.lower() in imdb_name_with_year_to_id.keys():
                    our_ids.append( imdb_name_with_year_to_id[movie_name.lower()] )

                elif year_flag == 0 and movie_name.lower() in imdb_name_without_year_to_id.keys():  
                    our_ids.append( imdb_name_without_year_to_id[movie_name.lower()] )
                    if movie_name.lower() in duplicate_name_set.keys():
                        suggested_is_in_duplicate += 1

                elif year_flag == 1 and movie_name.split(' (')[0].lower() in imdb_name_without_year_to_id.keys():
                    our_ids.append( imdb_name_without_year_to_id[movie_name.split(' (')[0].lower()] )
                    if movie_name.split(' (')[0].lower() in duplicate_name_set.keys():
                        suggested_is_in_duplicate += 1
                else:
                    not_exist_suggested_movie_db[movie_name]=1
                    suggest_not_exist_count += 1
#                     our_ids = None

            if len(recommended_list) > 1:
                multi_rec_count += 1
        else:
            our_ids = None
        if our_ids == []: our_ids = None
        this_dialog_post.append((item, our_ids))
    
    all_data_test.append(this_dialog_post)

Loaded 1342 test conversations


In [29]:
print(len(not_exist_suggested_movie_db), suggest_not_exist_count)
print(rec_count, multi_rec_count)
print(suggested_is_in_duplicate)

52 179
0 1747
154


In [30]:
print(len(not_exist_suggested_movie_db), suggest_not_exist_count)
print(rec_count, multi_rec_count)
print(suggested_is_in_duplicate)

52 179
0 1747
154


In [31]:
all_data_test[108]

[('A: Hello have you seen [MOVIE_ID] ? It is one of my favorite movies [MOVIE_ID]',
  [28381, 28381]),
 ('B: Hi yes, that is a classic! I love movies like that Also, [MOVIE_ID]',
  [25300]),
 ('A: all the mob movies are great like [MOVIE_ID]', [28769]),
 ('B: another one is [MOVIE_ID] I tend to enjoy gangster movies', [28769]),
 ('A: nice me too', None),
 ('B: yes!!', None),
 ('A: have you seen the [MOVIE_ID] ? Its the ultimate classic', [43116]),
 ('B: I actually hated [MOVIE_ID]  and never watched any of the others.  It was just too slow lol',
  [15528]),
 ('A: they do take some time to finish, theyre all 3 hours plus', None),
 ('B: Any other good suggestions? Like the movie [MOVIE_ID] for me that was another classic',
  [30671]),
 ("A: if you haven't seen it yet, [MOVIE_ID] is great", [45462]),
 ('B: Thanks, sounds good.', None),
 ('A: thoes are all good', None),
 ('B: Take care!', None),
 ('A: thanks', None)]

## train data

In [32]:
train_data = []
for line in open("./website-data/redial_dataset/train_data.jsonl", "r"):
    train_data.append(json.loads(line))
print("Loaded {} train conversations".format(len(train_data)))

all_data_train = [] #result

for i in range(len(train_data)):
    temp = train_data[i]
    mentioned_movies = temp['movieMentions']
    status_movies = temp['respondentQuestions']
    messages = temp['messages']
    
    # A is recommender B is customer
    AB_dict = {}
    AB_dict[temp['respondentWorkerId']] = 'A'
    AB_dict[temp['initiatorWorkerId']] = 'B'

    # dialog temp
    this_dialog_tmp = []
    for j in range(len(messages)):
        message = messages[j]
        userID = message['senderWorkerId']
        text = message['text']
        this_dialog_tmp.append([userID, text])

    # merge consecutive sentences from the same user into one sentence
    this_dialog = []
    j = 0
    while j < len(this_dialog_tmp):
        userID = this_dialog_tmp[j][0]
        text = this_dialog_tmp[j][1]
        k = 1
        while j + k < len(this_dialog_tmp) and userID == this_dialog_tmp[j+k][0]:
            text = text + ' ' + this_dialog_tmp[j+k][1]
            k += 1
        j = j + k   
        this_dialog.append(AB_dict[userID] + ": " + text) 
        
    # extract @movie ID, check which is recommendaiton
    this_dialog_post = []

    for item in this_dialog:
        this_setence_movie_id = re.findall(r"@\d+", item)
        recommended_list = []
        for m in this_setence_movie_id:
            m_id = m.split('@')[1]
            
            if len(status_movies) > 0 and m_id in status_movies.keys():# and status_movies[m_id]['suggested']==1:
                recommended_list.append(m_id)
            
            # replace id with name in the dialog
            if m_id in mentioned_movies.keys():
                movie_name = mentioned_movies[m_id]
                movie_name, year_flag = refine_movie_name(movie_name)
#                 item = item.replace(str(m), movie_name)
                item = item.replace(str(m), "[MOVIE_ID]") # use placeholder
        
        our_ids = []
        if len(recommended_list) > 0:
            for rec_id in range(len(recommended_list)):
                rec_count += 1
                movie_name = mentioned_movies[recommended_list[rec_id]]
                movie_name, year_flag = refine_movie_name(movie_name)
                if year_flag == 1 and movie_name.lower() in imdb_name_with_year_to_id.keys():
                    our_ids.append( imdb_name_with_year_to_id[movie_name.lower()] )

                elif year_flag == 0 and movie_name.lower() in imdb_name_without_year_to_id.keys():  
                    our_ids.append( imdb_name_without_year_to_id[movie_name.lower()] )
                    if movie_name.lower() in duplicate_name_set.keys():
                        suggested_is_in_duplicate += 1

                elif year_flag == 1 and movie_name.split(' (')[0].lower() in imdb_name_without_year_to_id.keys():
                    our_ids.append( imdb_name_without_year_to_id[movie_name.split(' (')[0].lower()] )
                    if movie_name.split(' (')[0].lower() in duplicate_name_set.keys():
                        suggested_is_in_duplicate += 1
                else:
                    not_exist_suggested_movie_db[movie_name]=1
                    suggest_not_exist_count += 1
#                     our_id = None
            
            if len(recommended_list) > 1:
                multi_rec_count += 1
        else:
            our_ids = None
        if our_ids == []: our_ids = None
        this_dialog_post.append((item, our_ids))
    
    all_data_train.append(this_dialog_post)

Loaded 10006 train conversations


In [33]:
print(len(not_exist_suggested_movie_db), suggest_not_exist_count)
print(rec_count, multi_rec_count)
print(suggested_is_in_duplicate)

179 1351
63669 14322
1692


In [34]:
print(len(not_exist_suggested_movie_db), suggest_not_exist_count)
print(rec_count, multi_rec_count)
print(suggested_is_in_duplicate)

179 1351
63669 14322
1692


In [35]:
all_data_train[12]

[('B: HI there~', None),
 ('A: Hi what kind of movies doe you like to watch?', None),
 ('B: I like mostly anything. Especially muscials and comedy Im not fond of movies like [MOVIE_ID]',
  [89461]),
 ('A: I love comedy movies have you seen [MOVIE_ID] it is very funny',
  [59079]),
 ('B: I have not seen that yet.Is that with the Rock&quot;', None),
 ('A: No it has Will Ferrel and Mark Wahlberg', None),
 ('B: Oh, ok, and they just made a second one too right?', None),
 ('A: [MOVIE_ID] has the rock and Kevin Hart you may like that Yeah they did it is suppose to be even funnier.',
  [58635]),
 ('B: I did see [MOVIE_ID] and liked that a lot!', [58635]),
 ("A: Then you should definitely see daddy's home", None),
 ('B: do you have one more suggestion before we go?', None),
 ('A: Yeah [MOVIE_ID] and [MOVIE_ID] were great Kevin Hart movies',
  [68914, 57792]),
 ('B: Great, I still have yet to see those. makes me laugh when he was on Conan with Ice Cube promoting htat thank you! we can submit no

In [36]:
cleaned_data_train = []

In [37]:
for data_train in all_data_train:
    data_point = []
    for u, r in data_train:
        if u[0] == "B":
            data_point.append((u, r))
        else:
            data_point.append((u, r))
    cleaned_data_train.append(data_point)

In [38]:
super_cleaned_data_train = []
for data_train in cleaned_data_train:
    valid = True
    for u, r in data_train: 
        if r == None: continue
        else:
            for item in r:
                if item not in metioned_movie_db.keys():
                    valid = False
                    break
    if valid:
        super_cleaned_data_train.append(data_train)
            

In [39]:
len(super_cleaned_data_train)

10006

In [40]:
cleaned_data_test = []
for data_test in all_data_test:
    data_point = []
    for u, r in data_test:
        if u[0] == "B":
            data_point.append((u, r))
        else:
            data_point.append((u, r))
    cleaned_data_test.append(data_point)
    
super_cleaned_data_test = []
for data_test in cleaned_data_test:
    valid = True
    for u, r in data_test: 
        if r == None: continue
        else:
            for item in r:
                if item not in metioned_movie_db.keys():
                    valid = False
                    break
    if valid:
        super_cleaned_data_test.append(data_test)

In [41]:
len(super_cleaned_data_test)

1342

In [42]:
movie_db_60000 = {}

In [43]:
for k, v in metioned_movie_db.items():
    movie_db_60000[k] = v
    all_movie_db.pop(k)

for k, v in all_movie_db.items():
    movie_db_60000[k] = v
    if len(movie_db_60000) >= 60000:
        break

In [44]:
len(movie_db_60000)

60000

In [45]:
torch.save(super_cleaned_data_train, "/local-scratch1/data/by2299/redial_full_train_placeholder")
torch.save(super_cleaned_data_test, "/local-scratch1/data/by2299/redial_full_test_placeholder")
torch.save(metioned_movie_db, "/local-scratch1/data/by2299/redial_full_movie_db_placeholder")